# Ants Bees VGG16 による解析

# Fitting 

# TypeBによるTrainingクラスの作成

In [ ]:
%pwd
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
'''
3. 前処理前後の画像を確認する
'''
import numpy as np
from PIL import Image
from torchvision import transforms
import matplotlib.pyplot as plt
import os
import torch
import torch.utils.data as data
from pathlib import Path


In [ ]:
data_dir = '/content/gdrive/My Drive/Colab Notebooks/AntsBees/data'
root_dir= '/content/gdrive/My Drive/Colab Notebooks/AntsBees/data/hymenoptera_data'

In [ ]:
os.chdir('/content/gdrive/My Drive/Colab Notebooks/AntsBees')

###Deviceの設定

In [ ]:
# 使用可能なデバイス(CPUまたはGPU）を取得する
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [ ]:
torch.cuda.synchronize()
print(torch.cuda.memory_allocated())

0


#(1) DataSet の作成

### train_list, val_list の作成

In [ ]:
import glob
import pprint

def make_datapath_list(phase='train'):
    '''
    データのファイルパスを格納したリストを作成する。

    Parameters:
      phase(str): 'train'または'val'

    Returns:
      path_list(list): 画像データのパスを格納したリスト
    '''
    # 画像ファイルのルートディレクトリ
    # 画像ファイルパスのフォーマットを作成
    # rootpath +
    #   train/ants/*.jpg
    #   train/bees/*.jpg
    #   val/ants/*.jpg
    #   val/bees/*.jpg
    tt= phase +'/**/*.jpg'
    target_path = os.path.join(root_dir,tt)
    # ファイルパスを格納するリスト
    path_list = []  # ここに格納する

    # glob()でファイルパスを取得してリストに追加
    for path in glob.glob(target_path):
        path_list.append(path)

    return path_list

# ファイルパスのリストを生成
train_list = make_datapath_list(phase='train')
val_list = make_datapath_list(phase='val')

print(train_list)

# 訓練データのファイルパスの前後5要素ずつ出力
print('train')
pprint.pprint(train_list[:5])
pprint.pprint(train_list[-6:-1])
# 検証データのファイルパスの前後5要素ずつ出力
print('val')
pprint.pprint(val_list[:5])
pprint.pprint(val_list[-6:-1])

['/content/gdrive/My Drive/Colab Notebooks/AntsBees/data/hymenoptera_data/train/ants/0013035.jpg', '/content/gdrive/My Drive/Colab Notebooks/AntsBees/data/hymenoptera_data/train/ants/1030023514_aad5c608f9.jpg', '/content/gdrive/My Drive/Colab Notebooks/AntsBees/data/hymenoptera_data/train/ants/1095476100_3906d8afde.jpg', '/content/gdrive/My Drive/Colab Notebooks/AntsBees/data/hymenoptera_data/train/ants/1099452230_d1949d3250.jpg', '/content/gdrive/My Drive/Colab Notebooks/AntsBees/data/hymenoptera_data/train/ants/116570827_e9c126745d.jpg', '/content/gdrive/My Drive/Colab Notebooks/AntsBees/data/hymenoptera_data/train/ants/1225872729_6f0856588f.jpg', '/content/gdrive/My Drive/Colab Notebooks/AntsBees/data/hymenoptera_data/train/ants/1262877379_64fcada201.jpg', '/content/gdrive/My Drive/Colab Notebooks/AntsBees/data/hymenoptera_data/train/ants/1269756697_0bce92cdab.jpg', '/content/gdrive/My Drive/Colab Notebooks/AntsBees/data/hymenoptera_data/train/ants/1286984635_5119e80de1.jpg', '/cont

In [ ]:
p=Path(train_list[5])
print(p.parts[-2])

p2=Path(val_list[-4])
print(p2.parts[-2])

ants
bees


#（２）DataSetの作成

In [ ]:
from dsets.dsets import MakeDataset
from util.ImageTransform import ImageTransform

In [ ]:
'''
6. データローダーの生成
'''
import torch

# ミニバッチのサイズを指定
batch_size = 10
SIZE = 224
# 標準化する際の各RGBの平均値
MEAN = (0.485, 0.456, 0.406) # ImageNetデータセットの平均値を使用
# 標準化する際の各RGBの標準偏差
STD = (0.229, 0.224, 0.225)  # ImageNetデータセットの標準偏差を使用


# 画像のサイズ、平均値、標準偏差の定数値
size, mean, std = SIZE, MEAN, STD

# MakeDatasetで前処理後の訓練データと正解ラベルを取得
train_dataset = MakeDataset(
    file_list=train_list, # 訓練データのファイルパス
    transform=ImageTransform(size, mean, std), # 前処理後のデータ
    phase='train')
# MakeDatasetで前処理後の検証データと正解ラベルを取得
val_dataset = MakeDataset(
    file_list=val_list, # 検証データのファイルパス
    transform=ImageTransform(size, mean, std), # 前処理後のデータ
    phase='val')


#(2) DataLoader の作成

In [ ]:
'''
6. データローダーの生成
'''
from torch.utils.data import DataLoader

# 訓練用のデータローダー:(バッチサイズ, 3, 224, 224)を生成
train_dl = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# 検証用のデータローダー:(バッチサイズ, 3, 224, 224)を生成
val_dl = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)



#(4) モデルの作成

In [ ]:
'''
7. 学習済みのVGG16モデルをロード
'''
from torchvision import models
import torch.nn as nn

# ImageNetで事前トレーニングされたVGG16モデルを取得
model = models.vgg16(pretrained=True)

# VGG16の出力層のユニット数を2にする
model.classifier[6] = nn.Linear(
    in_features=4096, # 入力サイズはデフォルトの4096
    out_features=2)   # 出力はデフォルトの1000から2に変更


model = model.to(device)
print(model)

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth


  0%|          | 0.00/528M [00:00<?, ?B/s]

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

# For Loop with DataLoaderの研究

In [ ]:
batch_iterator = iter(train_dl)
batch_tup = next(batch_iterator)


input_t, label_t = batch_tup

input_g=input_t.to(device)
label_g=label_t.to(device)
model = model.to(device)

outputs = model(input_g)

pred_label= outputs.argmax(dim=-1)
print(pred_label)
pred_label_list = pred_label.tolist()

print(pred_label_list)

print(outputs)
print(outputs.argmax(dim=-1).tolist())
print(label_t.tolist())
from sklearn.metrics import accuracy_score
print(accuracy_score(label_t.tolist(),pred_label.tolist()))



tensor([0, 0, 1, 0, 0, 0, 0, 0, 0, 0], device='cuda:0')
[0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
tensor([[ 0.4467,  0.2024],
        [ 0.4045, -0.1849],
        [-0.3361,  0.5445],
        [ 0.3498, -0.8817],
        [-0.1117, -0.2227],
        [ 0.2895, -0.6828],
        [ 0.1060, -0.3417],
        [-0.2249, -1.1343],
        [ 1.0833, -0.4194],
        [ 1.1258, -0.5730]], device='cuda:0', grad_fn=<AddmmBackward0>)
[0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
[0, 1, 0, 1, 1, 1, 0, 1, 1, 1]
0.2


In [ ]:
softmax = nn.Softmax(dim=1)
prob_g=softmax(outputs)
print(prob_g[:,1])

tensor([0.4392, 0.3568, 0.7069, 0.2259, 0.4723, 0.2744, 0.3899, 0.2871, 0.1820,
        0.1546], device='cuda:0', grad_fn=<SelectBackward0>)


In [ ]:
loss_func = nn.CrossEntropyLoss(reduction='none')
loss_g = loss_func(outputs,label_g)
print(loss_g)

tensor([0.5784, 1.0306, 1.2274, 1.4876, 0.7502, 1.2931, 0.4941, 1.2478, 1.7037,
        1.8668], device='cuda:0', grad_fn=<NllLossBackward0>)


In [ ]:
print(outputs)

tensor([[ 0.4467,  0.2024],
        [ 0.4045, -0.1849],
        [-0.3361,  0.5445],
        [ 0.3498, -0.8817],
        [-0.1117, -0.2227],
        [ 0.2895, -0.6828],
        [ 0.1060, -0.3417],
        [-0.2249, -1.1343],
        [ 1.0833, -0.4194],
        [ 1.1258, -0.5730]], device='cuda:0', grad_fn=<AddmmBackward0>)


In [ ]:

batch_iterator = iter(train_dl)
batch_tup = next(batch_iterator)

metrics_g= torch.zeros(3,len(train_dl.dataset),device='cuda',)

input_t, label_t = batch_tup

input_g = input_t.to('cuda')
label_g = label_t.to('cuda')

outputs  = model(input_g)

softmax = nn.Softmax(dim=1)
prob_g= softmax(outputs)
#pred_label_g= outputs.argmax(dim=-1)
#print(prob_g[:,1])

loss_func = nn.CrossEntropyLoss(reduction='none')
loss_g = loss_func(outputs,label_g)

start_ndx = 0 * batch_size
end_ndx = start_ndx + label_t.size()[0]
print('start_ndx,end_ndx',start_ndx,end_ndx,label_g.shape[0],prob_g.shape[0],loss_g.shape[0])

metrics_g[0, start_ndx:end_ndx] = label_g
metrics_g[1, start_ndx:end_ndx] = prob_g[:,1]
metrics_g[2, start_ndx:end_ndx] = loss_g

print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
print('Cached:   ', round(torch.cuda.memory_reserved(0)/1024**3,1), 'GB')

print(metrics_g[:,0:10])

import sys

print("{}{: >25}{}{: >10}{}".format('|','Variable Name','|','Memory','|'))
print(" ------------------------------------ ")
for var_name in dir():
    if not var_name.startswith("_"):
        print("{}{: >25}{}{: >10}{}".format('|',var_name,'|',sys.getsizeof(eval(var_name)),'|'))

start_ndx,end_ndx 0 10 10 10 10
Allocated: 1.9 GB
Cached:    2.2 GB
tensor([[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 1.0000, 1.0000, 0.0000,
         0.0000],
        [0.5888, 0.3774, 0.3907, 0.4094, 0.3272, 0.2734, 0.1310, 0.2081, 0.3340,
         0.4163],
        [0.8887, 0.4738, 0.4955, 0.5266, 0.3964, 0.3194, 2.0326, 1.5696, 0.4065,
         0.5384]], device='cuda:0', grad_fn=<SliceBackward0>)
|            Variable Name|    Memory|
 ------------------------------------ 
|               DataLoader|      1472|
|                    Image|        88|
|           ImageTransform|      1064|
|                       In|       272|
|                     MEAN|        80|
|              MakeDataset|      1064|
|                      Out|       248|
|                     Path|       896|
|                     SIZE|        28|
|                      STD|        80|
|           accuracy_score|       144|
|           batch_iterator|        64|
|               batch_size|        28|
|     

In [ ]:

batch_iterator = iter(train_dl)
batch_tup = next(batch_iterator)

#metrics_g= torch.zeros(3,len(train_dl.dataset),device='cuda',)

input_t, label_t = batch_tup

input_g = input_t.to('cuda')
label_g = label_t.to('cuda')

outputs  = model(input_g)

softmax = nn.Softmax(dim=1)
prob_g= softmax(outputs)
#pred_label_g= outputs.argmax(dim=-1)
#print(prob_g[:,1])

loss_func = nn.CrossEntropyLoss(reduction='none')
loss_g = loss_func(outputs,label_g)

start_ndx = 1 * batch_size
end_ndx = start_ndx + label_t.size()[0]
print('start_ndx,end_ndx',start_ndx,end_ndx,label_g.shape[0],prob_g.shape[0],loss_g.shape[0])

metrics_g[0, start_ndx:end_ndx] = label_g
metrics_g[1, start_ndx:end_ndx] = prob_g[:,1]
metrics_g[2, start_ndx:end_ndx] = loss_g

print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
print('Cached:   ', round(torch.cuda.memory_reserved(0)/1024**3,1), 'GB')

print(metrics_g[:,0:20])

import sys

print("{}{: >25}{}{: >10}{}".format('|','Variable Name','|','Memory','|'))
print(" ------------------------------------ ")
for var_name in dir():
    if not var_name.startswith("_"):
        print("{}{: >25}{}{: >10}{}".format('|',var_name,'|',sys.getsizeof(eval(var_name)),'|'))



start_ndx,end_ndx 10 20 10 10 10
Allocated: 1.9 GB
Cached:    2.2 GB
tensor([[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 1.0000, 1.0000, 0.0000,
         0.0000, 1.0000, 0.0000, 0.0000, 0.0000, 1.0000, 0.0000, 1.0000, 1.0000,
         1.0000, 1.0000],
        [0.5888, 0.3774, 0.3907, 0.4094, 0.3272, 0.2734, 0.1310, 0.2081, 0.3340,
         0.4163, 0.1667, 0.5482, 0.6373, 0.6197, 0.2603, 0.2665, 0.2684, 0.5109,
         0.4352, 0.1244],
        [0.8887, 0.4738, 0.4955, 0.5266, 0.3964, 0.3194, 2.0326, 1.5696, 0.4065,
         0.5384, 1.7914, 0.7945, 1.0142, 0.9667, 1.3458, 0.3099, 1.3153, 0.6716,
         0.8319, 2.0839]], device='cuda:0', grad_fn=<SliceBackward0>)
|            Variable Name|    Memory|
 ------------------------------------ 
|               DataLoader|      1472|
|                    Image|        88|
|           ImageTransform|      1064|
|                       In|       272|
|                     MEAN|        80|
|              MakeDataset|      1064|
|          

In [ ]:
var_name = 'metrics_g'
sys.getsizeof(eval(var_name))

88

In [ ]:

batch_iterator = iter(train_dl)
batch_tup = next(batch_iterator)

#metrics_g= torch.zeros(3,len(train_dl.dataset),device='cuda',)

input_t, label_t = batch_tup

input_g = input_t.to('cuda')
label_g = label_t.to('cuda')

outputs  = model(input_g)

softmax = nn.Softmax(dim=1)
prob_g= softmax(outputs)
#pred_label_g= outputs.argmax(dim=-1)
#print(prob_g[:,1])

loss_func = nn.CrossEntropyLoss(reduction='none')
loss_g = loss_func(outputs,label_g)

start_ndx = 2 * batch_size
end_ndx = start_ndx + label_t.size()[0]
print('start_ndx,end_ndx',start_ndx,end_ndx,label_g.shape[0],prob_g.shape[0],loss_g.shape[0])

metrics_g[0, start_ndx:end_ndx] = label_g
metrics_g[1, start_ndx:end_ndx] = prob_g[:,1]
metrics_g[2, start_ndx:end_ndx] = loss_g

print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
print('Cached:   ', round(torch.cuda.memory_reserved(0)/1024**3,1), 'GB')

print(metrics_g[:,0:30])

import sys

print("{}{: >25}{}{: >10}{}".format('|','Variable Name','|','Memory','|'))
print(" ------------------------------------ ")
for var_name in dir():
    if not var_name.startswith("_"):
        print("{}{: >25}{}{: >10}{}".format('|',var_name,'|',sys.getsizeof(eval(var_name)),'|'))

start_ndx,end_ndx 20 30 10 10 10
Allocated: 2.6 GB
Cached:    2.9 GB
tensor([[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 1.0000, 1.0000, 0.0000,
         0.0000, 1.0000, 0.0000, 0.0000, 0.0000, 1.0000, 0.0000, 1.0000, 1.0000,
         1.0000, 1.0000, 0.0000, 1.0000, 0.0000, 1.0000, 0.0000, 1.0000, 1.0000,
         0.0000, 1.0000, 1.0000],
        [0.5888, 0.3774, 0.3907, 0.4094, 0.3272, 0.2734, 0.1310, 0.2081, 0.3340,
         0.4163, 0.1667, 0.5482, 0.6373, 0.6197, 0.2603, 0.2665, 0.2684, 0.5109,
         0.4352, 0.1244, 0.2199, 0.3350, 0.4436, 0.4723, 0.2795, 0.4770, 0.4118,
         0.6801, 0.4125, 0.3516],
        [0.8887, 0.4738, 0.4955, 0.5266, 0.3964, 0.3194, 2.0326, 1.5696, 0.4065,
         0.5384, 1.7914, 0.7945, 1.0142, 0.9667, 1.3458, 0.3099, 1.3153, 0.6716,
         0.8319, 2.0839, 0.2484, 1.0935, 0.5862, 0.7502, 0.3278, 0.7402, 0.8872,
         1.1396, 0.8855, 1.0452]], device='cuda:0', grad_fn=<SliceBackward0>)
|            Variable Name|    Memory|
 ----------------

In [ ]:

batch_iterator = iter(train_dl)
batch_tup = next(batch_iterator)

#metrics_g= torch.zeros(3,len(train_dl.dataset),device='cuda',)

input_t, label_t = batch_tup

input_g = input_t.to('cuda')
label_g = label_t.to('cuda')

outputs  = model(input_g)

softmax = nn.Softmax(dim=1)
prob_g= softmax(outputs)
#pred_label_g= outputs.argmax(dim=-1)
#print(prob_g[:,1])

loss_func = nn.CrossEntropyLoss(reduction='none')
loss_g = loss_func(outputs,label_g)

start_ndx = 3 * batch_size
end_ndx = start_ndx + label_t.size()[0]
print('start_ndx,end_ndx',start_ndx,end_ndx,label_g.shape[0],prob_g.shape[0],loss_g.shape[0])

metrics_g[0, start_ndx:end_ndx] = label_g
metrics_g[1, start_ndx:end_ndx] = prob_g[:,1]
metrics_g[2, start_ndx:end_ndx] = loss_g

print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
print('Cached:   ', round(torch.cuda.memory_reserved(0)/1024**3,1), 'GB')

print(metrics_g[:,0:40])

import sys

print("{}{: >25}{}{: >10}{}".format('|','Variable Name','|','Memory','|'))
print(" ------------------------------------ ")
for var_name in dir():
    if not var_name.startswith("_"):
        print("{}{: >25}{}{: >10}{}".format('|',var_name,'|',sys.getsizeof(eval(var_name)),'|'))

start_ndx,end_ndx 30 40 10 10 10
Allocated: 3.2 GB
Cached:    3.6 GB
tensor([[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 1.0000, 1.0000, 0.0000,
         0.0000, 1.0000, 0.0000, 0.0000, 0.0000, 1.0000, 0.0000, 1.0000, 1.0000,
         1.0000, 1.0000, 0.0000, 1.0000, 0.0000, 1.0000, 0.0000, 1.0000, 1.0000,
         0.0000, 1.0000, 1.0000, 0.0000, 1.0000, 0.0000, 1.0000, 0.0000, 1.0000,
         1.0000, 1.0000, 1.0000, 1.0000],
        [0.5888, 0.3774, 0.3907, 0.4094, 0.3272, 0.2734, 0.1310, 0.2081, 0.3340,
         0.4163, 0.1667, 0.5482, 0.6373, 0.6197, 0.2603, 0.2665, 0.2684, 0.5109,
         0.4352, 0.1244, 0.2199, 0.3350, 0.4436, 0.4723, 0.2795, 0.4770, 0.4118,
         0.6801, 0.4125, 0.3516, 0.7554, 0.3029, 0.8198, 0.2020, 0.8531, 0.2752,
         0.2361, 0.1572, 0.4978, 0.5666],
        [0.8887, 0.4738, 0.4955, 0.5266, 0.3964, 0.3194, 2.0326, 1.5696, 0.4065,
         0.5384, 1.7914, 0.7945, 1.0142, 0.9667, 1.3458, 0.3099, 1.3153, 0.6716,
         0.8319, 2.0839, 0.2484, 1.09

In [ ]:
print(metrics_g[:,:40])

tensor([[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 1.0000, 1.0000, 0.0000,
         0.0000, 1.0000, 0.0000, 0.0000, 0.0000, 1.0000, 0.0000, 1.0000, 1.0000,
         1.0000, 1.0000, 0.0000, 1.0000, 0.0000, 1.0000, 0.0000, 1.0000, 1.0000,
         0.0000, 1.0000, 1.0000, 0.0000, 1.0000, 0.0000, 1.0000, 0.0000, 1.0000,
         1.0000, 1.0000, 1.0000, 1.0000],
        [0.5888, 0.3774, 0.3907, 0.4094, 0.3272, 0.2734, 0.1310, 0.2081, 0.3340,
         0.4163, 0.1667, 0.5482, 0.6373, 0.6197, 0.2603, 0.2665, 0.2684, 0.5109,
         0.4352, 0.1244, 0.2199, 0.3350, 0.4436, 0.4723, 0.2795, 0.4770, 0.4118,
         0.6801, 0.4125, 0.3516, 0.7554, 0.3029, 0.8198, 0.2020, 0.8531, 0.2752,
         0.2361, 0.1572, 0.4978, 0.5666],
        [0.8887, 0.4738, 0.4955, 0.5266, 0.3964, 0.3194, 2.0326, 1.5696, 0.4065,
         0.5384, 1.7914, 0.7945, 1.0142, 0.9667, 1.3458, 0.3099, 1.3153, 0.6716,
         0.8319, 2.0839, 0.2484, 1.0935, 0.5862, 0.7502, 0.3278, 0.7402, 0.8872,
         1.1396, 0.8855, 